# Install Roboflow

In [11]:
!pip install roboflow

# Load Roboflow API


In [12]:

# we are trying to import the roboflow library
from roboflow import Roboflow

# connect it with roboflow account through by using the API_Key
API_key = Roboflow(api_key="Da5RnYli8ec7tzzyU3BK")

# now its connect with roboflow account now we can access from the following workspace which have present the following project
project = API_key.workspace("mohamed-traore-2ekkp").project("chest-x-rays-qjmia")

loading Roboflow workspace...
loading Roboflow project...


# Download Dataset

In [13]:
dataset = project.version(4).download("folder")

# Make Directories 

In [14]:
train_direc = "/kaggle/working/Chest-X-Rays-4/train"

valid_direc = "/kaggle/working/Chest-X-Rays-4/valid"

test_direc = "/kaggle/working/Chest-X-Rays-4/test"

# DataGenerator (Settings of Data)

In [15]:
# import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras


"""
1) Basically we can only apply the Augumentation on training data to learn the better and generalize 
   things instead of learning exactly specific pattern which cause of overfitting (memorization).

2) We use the keras augumentation for images to increase its numbers of images. because manually
   collection of images are very hardly and also costly

3) Some time wrong augumenation also create problem of noise data which make the model confuse
   and also reduce its performance

"""


#__________________Data Augumentation Settings____________________

train_datagen = ImageDataGenerator(
    
   rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


#_________________Validation Settings____________________

valid_datagen = ImageDataGenerator(rescale=1.0/255)

#_______________Test Data Settings______________________

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Apply the Settings to Make New Data Files


In [16]:
#  Now Use the above settings and apply it on training, validation and testing 

# training     = augumentation
# validation   = scaling
# testing      = scaling

#_________Train_Data______

training_data = train_datagen.flow_from_directory(
    train_direc,  
    target_size = (224, 224),
    batch_size  = 32,
    class_mode   ="binary",
)

#_______Valid_Data______

validation_data = valid_datagen.flow_from_directory(
    valid_direc,
    target_size = (224, 224),
    batch_size  = 32,
    class_mode  = "binary"
)

#______Test_Data______

testing_data = test_datagen.flow_from_directory(
    test_direc,
    target_size = (224, 224),
    batch_size  = 32,
    class_mode  = "binary"
)


Found 12229 images belonging to 2 classes.
Found 1165 images belonging to 2 classes.
Found 582 images belonging to 2 classes.


# Build CNNs

In [17]:
import tensorflow

from tensorflow import keras

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

from keras.callbacks import EarlyStopping

model = Sequential()

#_______Input_Layer_________

model.add(Conv2D(16, kernel_size=(2,2), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2), padding='same'))
model.add(BatchNormalization())

#______Hidden_Layers_________

model.add(Conv2D(32, kernel_size=(2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(512, kernel_size=(2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(1024, kernel_size=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4), strides=(2,2), padding='same'))
model.add(BatchNormalization())


#______Flatten_Layer_______

model.add(Flatten())


#______Fully_Connected_Layers________

model.add(Dense(units=256, activation='relu'))

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=64, activation='relu'))


#_____Output_Layer_______

model.add(Dense(units=1, activation='sigmoid'))

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


# Model Summary

In [18]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)                    │ (None, 223, 223, 16)        │             208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 112, 112, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 112, 112, 16)        │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 111, 111, 32)        │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 56, 56, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 56, 56, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 55, 55, 64)          │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 28, 28, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 27, 27, 128)         │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 14, 14, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 13, 13, 256)         │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 7, 7, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 7, 7, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 6, 6, 512)           │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 3, 3, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 3, 3, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 2,323,057 (8.86 MB)

 Trainable params: 2,318,993 (8.85 MB)

 Non-trainable params: 4,064 (15.88 KB)

# Compile Model

In [19]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping

In [25]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",              # kis metric ko monitor karna hai (jaise 'val_loss' ya 'val_accuracy')
    min_delta=0.0001,                     # itna change aana chahiye metric mein, warna improvement nahi maana jaayega
    patience=10,                      # kitne epochs tak wait kare bina improvement ke
    verbose=1,                       # 1 karo to message print karega jab stop kare
    mode="auto",                     # 'min', 'max' ya 'auto' - loss ke liye 'min', accuracy ke liye 'm                   # agar diya gaya hai, to baseline se below jaate hi stop kar do
    restore_best_weights=True,      # best weights restore kare ya nahi
    start_from_epoch=10               # kis epoch ke baad monitoring start kare
)


# Train Model

In [26]:
model.fit(training_data, epochs=100, validation_data=validation_data, callbacks=[early_stopping])

Epoch 1/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 128s 327ms/step - accuracy: 0.8957 - loss: 0.2672 - val_accuracy: 0.7442 - val_loss: 1.0584
Epoch 2/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 126s 323ms/step - accuracy: 0.9143 - loss: 0.2245 - val_accuracy: 0.9082 - val_loss: 0.2210
Epoch 3/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 126s 322ms/step - accuracy: 0.9296 - loss: 0.1887 - val_accuracy: 0.8369 - val_loss: 0.3650
Epoch 4/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 127s 324ms/step - accuracy: 0.9375 - loss: 0.1766 - val_accuracy: 0.9288 - val_loss: 0.1806
Epoch 5/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 128s 328ms/step - accuracy: 0.9363 - loss: 0.1732 - val_accuracy: 0.8129 - val_loss: 0.4390
Epoch 6/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 128s 328ms/step - accuracy: 0.9402 - loss: 0.1601 - val_accuracy: 0.9605 - val_loss: 0.1626
Epoch 7/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 127s 324ms/step - accuracy: 0.9517 - loss: 0.1359 - val_accuracy: 0.9622 - val_loss: 0.1044
Epoch 8/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 126s 323ms/step - accuracy: 

# Save Model

In [56]:
save_trained_model = model.save("Pneumonia_Detector.h5")


# Test Model 

In [57]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os


In [59]:
model = load_model('Pneumonia_Detector.h5')


In [64]:
img_path = '/kaggle/working/Chest-X-Rays-4/test/NORMAL/IM-0001-0001_jpeg.rf.f50cf8d271cda12e85390654010d20df.jpg'  # change to your actual test image path

# Load image with same size used in training (e.g., 64x64)
img = image.load_img(img_path, target_size=(224, 224))  

# Convert image to array
img_array = image.img_to_array(img)

# Expand dimensions (for batch)
img_array = np.expand_dims(img_array, axis=0)

# Normalize (if trained on normalized images)
img_array = img_array / 255.0


In [65]:
prediction = model.predict(img_array)
print("Raw prediction:", prediction)

# Optional: convert to class label
predicted_class = np.argmax(prediction, axis=1)
print("Predicted class index:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Raw prediction: [[6.361773e-05]]
Predicted class index: [0]
